In [1]:
import feature_engine.encoding
import feature_engine.pipeline
import feature_engine.selection
import feature_engine.wrappers
import lightning
import pandas
import pycaret.classification as classification
import sklearn.preprocessing
import torch
import torchmetrics

In [2]:
lightning.seed_everything(0)

Seed set to 0


0

In [3]:
train_df = pandas.read_csv("train.csv")
train_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [4]:
pipeline = feature_engine.pipeline.make_pipeline(
    feature_engine.selection.DropFeatures(features_to_drop=["id", "CustomerId", "Surname"]),
    feature_engine.wrappers.SklearnTransformerWrapper(transformer=sklearn.preprocessing.MinMaxScaler(), variables=["CreditScore", "Age", "Tenure", "Balance", "EstimatedSalary"]),
    feature_engine.encoding.OrdinalEncoder(encoding_method='arbitrary', variables=["Geography", "Gender"]),
)

In [5]:
train_transformed_df = pipeline.fit_transform(train_df)
train_transformed_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.636,0,0,0.202703,0.3,0.000000,2,1.0,0.0,0.907279,0
1,0.554,0,0,0.202703,0.1,0.000000,2,1.0,1.0,0.247483,0
2,0.656,0,0,0.297297,1.0,0.000000,2,1.0,0.0,0.924364,0
3,0.462,0,0,0.216216,0.2,0.593398,1,1.0,1.0,0.422787,0
4,0.732,1,0,0.202703,0.5,0.000000,2,1.0,1.0,0.075293,0
...,...,...,...,...,...,...,...,...,...,...,...
165029,0.634,1,1,0.202703,0.2,0.000000,1,1.0,1.0,0.659179,0
165030,0.884,0,0,0.229730,0.3,0.000000,1,0.0,0.0,0.659177,0
165031,0.430,0,0,0.175676,0.5,0.000000,1,1.0,1.0,0.637151,0
165032,0.408,1,1,0.162162,0.7,0.643819,1,0.0,1.0,0.355841,0


In [6]:
test_df = pandas.read_csv("test.csv")
test_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110018,275052,15662091,P'eng,570,Spain,Male,29.0,7,116099.82,1,1.0,1.0,148087.62
110019,275053,15774133,Cox,575,France,Female,36.0,4,178032.53,1,1.0,1.0,42181.68
110020,275054,15728456,Ch'iu,712,France,Male,31.0,2,0.00,2,1.0,0.0,16287.38
110021,275055,15687541,Yegorova,709,France,Female,32.0,3,0.00,1,1.0,1.0,158816.58


In [7]:
test_df["Exited"] = 0
test_transformed_df = pipeline.transform(test_df)
test_transformed_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.472,0,1,0.067568,0.2,0.000000,2,0.0,1.0,0.804903,0
1,0.666,0,1,0.378378,0.2,0.000000,1,1.0,0.0,0.362723,0
2,0.612,0,1,0.216216,0.7,0.000000,2,1.0,0.0,0.694419,0
3,0.662,0,0,0.243243,0.8,0.000000,1,1.0,0.0,0.569654,0
4,0.804,2,0,0.270270,1.0,0.483318,1,1.0,0.0,0.697164,0
...,...,...,...,...,...,...,...,...,...,...,...
110018,0.440,1,0,0.148649,0.7,0.462737,1,1.0,1.0,0.740451,0
110019,0.450,0,1,0.243243,0.4,0.709581,1,1.0,1.0,0.210871,0
110020,0.724,0,0,0.175676,0.2,0.000000,2,1.0,0.0,0.081387,0
110021,0.718,0,1,0.189189,0.3,0.000000,1,1.0,1.0,0.794101,0


In [8]:
class MyModule(lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._stack = torch.nn.Sequential(
            torch.nn.Linear(10, 32), torch.nn.ReLU(),
            torch.nn.Linear(32, 2), torch.nn.Softmax(dim=1),
        )
        self._accuracy = torchmetrics.classification.BinaryAccuracy()

    def forward(self, x):
        return self._stack(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def training_step(self, batch, batch_idx):
        (loss, acc) = self._compute(batch, batch_idx)
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        (loss, acc) = self._compute(batch, batch_idx)
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        return loss

    def test_step(self, batch, batch_idx):
        (loss, acc) = self._compute(batch, batch_idx)
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return loss
    
    def _compute(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        pred = y_hat.argmax(axis=1)
        # print(f"{y=}")
        # print(f"{y_hat=}")
        # print(f"{pred=}")
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        acc = self._accuracy(pred, y)
        return (loss, acc)

In [9]:
class MyDataModule(lightning.LightningDataModule):
    def __init__(self):
        super().__init__()
        train_X_df = train_transformed_df.drop(columns="Exited")
        train_y_df = train_transformed_df["Exited"]
        train_X_tensor = torch.tensor(train_X_df.values, dtype=torch.float32)
        train_y_tensor = torch.tensor(train_y_df.values, dtype=torch.int64)
        all_dataset = torch.utils.data.TensorDataset(train_X_tensor, train_y_tensor)
        n_train = int(len(all_dataset) * 0.8)
        n_val = int(len(all_dataset) * 0.1)
        n_test = len(all_dataset) - n_train - n_val
        self._train_dataset, self._val_dataset, self._test_dataset = torch.utils.data.random_split(all_dataset, [n_train, n_val, n_test])
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self._train_dataset, batch_size=1024)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self._val_dataset, batch_size=1024)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self._test_dataset, batch_size=1024)

In [10]:
trainer = lightning.Trainer(max_epochs=200, callbacks=[lightning.pytorch.callbacks.early_stopping.EarlyStopping(monitor="val_loss", mode="min")])
model = MyModule()
loader = MyDataModule()
trainer.fit(model, datamodule=loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type           | Params | Mode 
-----------------------------------------------------
0 | _stack    | Sequential     | 418    | train
1 | _accuracy | BinaryAccuracy | 0      | train
-----------------------------------------------------
418       Trainable params
0         Non-trainable params
418       Total params
0.002     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                               | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=200` reached.


In [11]:
trainer.test(datamodule=loader)

Restoring states from the checkpoint path at C:\Users\hsmtk\repo\fantastic-octo-dollop\playground-series-s4e1\lightning_logs\version_16\checkpoints\epoch=199-step=25800.ckpt
Loaded model weights from the checkpoint at C:\Users\hsmtk\repo\fantastic-octo-dollop\playground-series-s4e1\lightning_logs\version_16\checkpoints\epoch=199-step=25800.ckpt


Testing: |                                                                                       | 0/? [00:00<…

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8620334267616272
        test_loss           0.44470056891441345
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.44470056891441345, 'test_acc': 0.8620334267616272}]

In [12]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

In [13]:
test_df = pandas.read_csv("test.csv")
test_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110018,275052,15662091,P'eng,570,Spain,Male,29.0,7,116099.82,1,1.0,1.0,148087.62
110019,275053,15774133,Cox,575,France,Female,36.0,4,178032.53,1,1.0,1.0,42181.68
110020,275054,15728456,Ch'iu,712,France,Male,31.0,2,0.00,2,1.0,0.0,16287.38
110021,275055,15687541,Yegorova,709,France,Female,32.0,3,0.00,1,1.0,1.0,158816.58


In [14]:
test_df["Exited"] = 0
test_transformed_df = pipeline.transform(test_df)
test_transformed_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.472,0,1,0.067568,0.2,0.000000,2,0.0,1.0,0.804903,0
1,0.666,0,1,0.378378,0.2,0.000000,1,1.0,0.0,0.362723,0
2,0.612,0,1,0.216216,0.7,0.000000,2,1.0,0.0,0.694419,0
3,0.662,0,0,0.243243,0.8,0.000000,1,1.0,0.0,0.569654,0
4,0.804,2,0,0.270270,1.0,0.483318,1,1.0,0.0,0.697164,0
...,...,...,...,...,...,...,...,...,...,...,...
110018,0.440,1,0,0.148649,0.7,0.462737,1,1.0,1.0,0.740451,0
110019,0.450,0,1,0.243243,0.4,0.709581,1,1.0,1.0,0.210871,0
110020,0.724,0,0,0.175676,0.2,0.000000,2,1.0,0.0,0.081387,0
110021,0.718,0,1,0.189189,0.3,0.000000,1,1.0,1.0,0.794101,0


In [19]:
test_without_target_df = test_transformed_df.drop(columns=["Exited"])
test_tensor = torch.tensor(test_without_target_df.values, dtype=torch.float32)
test_tensor.shape

torch.Size([110023, 10])

In [20]:
with torch.no_grad():
    predicted = model(test_tensor)

In [22]:
predicted.argmax(axis=1)

tensor([0, 1, 0,  ..., 0, 0, 0])